In [2]:
import numpy as np
import pandas as pd

Loads title.akas.tsv data

In [3]:
title_akas = pd.read_csv('/Users/seankamano/Downloads/title.akas.tsv', delimiter = '\t', encoding = 'utf-8')
print title_akas.head(10)

/Users/seankamano/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


     titleId  ordering                      title region language  \
0  tt0000001         1  Carmencita - spanyol tánc     HU       \N   
1  tt0000001         2                 Карменсита     RU       \N   
2  tt0000001         3                 Carmencita     US       \N   
3  tt0000001         4                 Carmencita     \N       \N   
4  tt0000002         1     Le clown et ses chiens     \N       \N   
5  tt0000002         2          A bohóc és kutyái     HU       \N   
6  tt0000002         3     Le clown et ses chiens     FR       \N   
7  tt0000002         4     The Clown and His Dogs     US       \N   
8  tt0000002         5         Клоун и его собаки     RU       \N   
9  tt0000003         1            Szegény Pierrot     HU       \N   

         types             attributes isOriginalTitle  
0  imdbDisplay                     \N               0  
1           \N                     \N               0  
2           \N                     \N               0  
3     original  

Need to load title.basics.tsv data to get movies only. Gets tconst for all movies.

In [4]:
title_basics = pd.read_csv('/Users/seankamano/Downloads/title.basics.tsv', delimiter='\t', encoding='utf-8')
title_basics_movie = title_basics[title_basics['titleType'] == 'movie']
uid = title_basics_movie['tconst']

print uid.head(10)

/Users/seankamano/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


8      tt0000009
145    tt0000147
332    tt0000335
499    tt0000502
571    tt0000574
611    tt0000615
626    tt0000630
669    tt0000675
670    tt0000676
673    tt0000679
Name: tconst, dtype: object


Get movies only from title.akas.tsv by taking uid and comparing them to the tconsts in title.akas.tsv. This filters out any entries that are not movies in title.akas.tsv

In [5]:
title_akas_movies = title_akas[title_akas['titleId'].isin(uid.values)]

print title_akas_movies.head(10)

        titleId  ordering                          title region language  \
46    tt0000009         1                     Miss Jerry     \N       \N   
47    tt0000009         2                     Miss Jerry     HU       \N   
48    tt0000009         3                     Miss Jerry     US       \N   
568   tt0000147         1  The Corbett-Fitzsimmons Fight     US       \N   
971   tt0000335         1        Early Christian Martyrs     AU       \N   
972   tt0000335         2          Soldiers of the Cross     AU       \N   
973   tt0000335         3          Soldiers of the Cross     \N       \N   
1447  tt0000502         1                       Bohemios     ES       \N   
1663  tt0000574         1     Kelly bandájának története     HU       \N   
1664  tt0000574         2    The Story of the Kelly Gang     \N       \N   

            types attributes isOriginalTitle  
46       original         \N               1  
47    imdbDisplay         \N               0  
48             \N     

Remove movies without any regional data. Store movies without regional data for later reference, if it's ever needed.

In [10]:
title_akas_movies_by_region = title_akas_movies[title_akas_movies['region'] != '\N']
title_akas_movies_no_region = title_akas_movies[title_akas_movies['region'] == '\N']
print title_akas_movies_by_region.head(10)
print '\n',"*"*100,'\n'
print title_akas_movies_no_region.head(10)

        titleId  ordering                           title region language  \
47    tt0000009         2                      Miss Jerry     HU       \N   
48    tt0000009         3                      Miss Jerry     US       \N   
568   tt0000147         1   The Corbett-Fitzsimmons Fight     US       \N   
971   tt0000335         1         Early Christian Martyrs     AU       \N   
972   tt0000335         2           Soldiers of the Cross     AU       \N   
1447  tt0000502         1                        Bohemios     ES       \N   
1663  tt0000574         1      Kelly bandájának története     HU       \N   
1665  tt0000574         3  Die Geschichte der Kelly Bande     DE       \N   
1666  tt0000574         4     The Story of the Kelly Gang     AU       \N   
1783  tt0000615         1              Robbery Under Arms     AU       \N   

            types attributes isOriginalTitle  
47    imdbDisplay         \N               0  
48             \N         \N               0  
568        

Need to make sense of region codes. Scrape data off of http://www.nationsonline.org/oneworld/country_code_list.htm

In [19]:
import urllib2
from bs4 import BeautifulSoup

soup = BeautifulSoup(urllib2.urlopen('http://www.nationsonline.org/oneworld/country_code_list.htm').read(), "lxml")

tableClassResults = soup.find("table",{"class":"tb86 rowstyle-alternate colstyle-alt no-arrow"})
countries = tableClassResults.find_all('td', {'class':'abs'})

def get_countries():
    for country in countries:
        try: 
            yield country.a.string
        except (AttributeError):
            try:
                yield country.em.string
            except (AttributeError):
                yield country.string
    
list_countries = [k for k in get_countries()]

print "Country names: ", list_countries[:10]

country_codes = tableClassResults.find_all('td', {'style':'text-align:center'})

def get_codes():
    for code in country_codes:
        if len(code.string) == 2 or code.string == ' AF':
            yield code.string

list_codes = [i for i in get_codes()]

print "Country codes: ", list_codes[:10]

Country names:  [u'Afghanistan', u'Aland Islands', u'Albania', u'Algeria', u'American Samoa', u'Andorra', u'Angola', u'Anguilla', u'Antarctica  \t  \t  \t', u'Antigua and Barbuda']
Country codes:  [u' AF', u'AX', u'AL', u'DZ', u'AS', u'AD', u'AO', u'AI', u'AQ', u'AG']


Need to change these codes into a form we can work with. Convert into strings and then zip with country names so we have a reference for what each two-character acronym represents.

In [29]:
from itertools import islice
for i in range(len(list_codes)):
    if str(list_codes[i]) != ' AF':
        list_codes[i] = str(list_codes[i])
    else:
        list_codes[i] = 'AF'

country_dict = dict(zip(list_codes,list_countries))
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

for key in country_dict:
    if key[0] == 'A':
        print key,country_dict[key]

AF Afghanistan
AW Aruba
AE United Arab Emirates
AD Andorra
AG Antigua and Barbuda
AI Anguilla
AM Armenia
AL Albania
AO Angola
AN Netherlands Antilles
AQ Antarctica  	  	  	
AS American Samoa
AR Argentina
AU Australia
AT Austria
AX Aland Islands
AZ Azerbaijan


We now a more recognizable form for each country code. Now to find how many movies have been released in each country. 

In [48]:
movies_per_country = dict(zip(list_codes, [0 for i in range(len(list_codes))]))
for code in title_akas_movies_by_region['region']:
    try:
        movies_per_country[str(code)] += 1
    except(KeyError):
        continue

for key in movies_per_country:
    if key[0] == 'A':
        print key,movies_per_country[key]

AF 77
AW 7
AE 129
AD 11
AG 11
AI 1
AM 184
AL 954
AO 36
AN 7
AQ 4
AS 2
AR 17519
AU 8740
AT 11654
AX 0
AZ 424


Now for the average rating of all movies seen by a given country. Need title.ratings.tsv, extract movies with regional codes from it, and then attach a rating to each title.

In [38]:
title_ratings = pd.read_csv('/Users/seankamano/Downloads/title.ratings.tsv', delimiter='\t', encoding='utf-8')
title_ratings.head(10)

,tconst,averageRating,numVotes
0,tt0000001,5.8,1350
1,tt0000002,6.5,157
2,tt0000003,6.6,933
3,tt0000004,6.4,93
4,tt0000005,6.2,1621
5,tt0000006,5.7,80
6,tt0000007,5.5,543
7,tt0000008,5.6,1441
8,tt0000009,5.4,62
9,tt0000010,6.9,4845


Get the tconsts in title.ratings.tsv

In [42]:
tconsts = title_ratings['tconst']
tconsts.head(10)

0    tt0000001
1    tt0000002
2    tt0000003
3    tt0000004
4    tt0000005
5    tt0000006
6    tt0000007
7    tt0000008
8    tt0000009
9    tt0000010
Name: tconst, dtype: object

Get all movies with a valid region and cross reference with movies which have an average rating.

In [64]:
movies_region_and_rating = title_akas_movies_by_region[title_akas_movies_by_region['titleId'].isin(tconsts.values)]
movies_region_and_rating.head(10)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
47,tt0000009,2,Miss Jerry,HU,\N,imdbDisplay,\N,0
48,tt0000009,3,Miss Jerry,US,\N,\N,\N,0
568,tt0000147,1,The Corbett-Fitzsimmons Fight,US,\N,\N,\N,0
971,tt0000335,1,Early Christian Martyrs,AU,\N,working,\N,0
972,tt0000335,2,Soldiers of the Cross,AU,\N,\N,\N,0
1663,tt0000574,1,Kelly bandájának története,HU,\N,imdbDisplay,\N,0
1665,tt0000574,3,Die Geschichte der Kelly Bande,DE,\N,\N,\N,0
1666,tt0000574,4,The Story of the Kelly Gang,AU,\N,\N,\N,0
1783,tt0000615,1,Robbery Under Arms,AU,\N,\N,\N,0
1820,tt0000630,1,Amleto,IT,\N,\N,\N,0


Need to get ratings and connect them to country codes. First get ratings from title.ratings.tsv and attach them with their respective tconsts. Then, by comparing tconsts with titleId in movies_region_and_rating, get the country code.

In [65]:
avg_rating_per_country = dict(zip(list_codes, [0 for i in range(len(list_codes))]))
titleIds = movies_region_and_rating['titleId']
ratings = title_ratings[title_ratings['tconst'].isin(titleIds.values)]
ratings.head(10)


,tconst,averageRating,numVotes
8,tt0000009,5.4,62
141,tt0000147,5.2,245
232,tt0000335,6.2,32
331,tt0000574,6.3,431
347,tt0000615,5.1,12
353,tt0000630,2.9,8
380,tt0000675,4.7,7
381,tt0000676,4.0,6
382,tt0000679,6.1,24
412,tt0000739,7.2,6
